In [7]:
%pip install xarray tornado_helper git+https://github.com/mit-ll/tornet.git huggingface_hub tensorflow[and-cuda] netcdf4 aria2 scikit-learn

  Cloning https://github.com/mit-ll/tornet.git to /tmp/pip-req-build-wgw7qwgt
  Running command git clone --filter=blob:none --quiet https://github.com/mit-ll/tornet.git /tmp/pip-req-build-wgw7qwgt
  Resolved https://github.com/mit-ll/tornet.git to commit 959eead659a2371e4bf7d7544165bf83c856b572
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os 

TORNET_ROOT = "./data_tornet"
GOES_ROOT   = "./data_goes"
YEARS = [2017, 2018, 2019, 2020, 2021, 2022]

os.environ['TORNET_ROOT'] = TORNET_ROOT
os.environ['GOES_ROOT'] = GOES_ROOT

In [2]:
# # Download Files 
# from tornado_helper import TorNet, GOES
# tornet = TorNet()
# tornet.download(YEARS)

# goes = GOES()
# goes.download(YEARS)

In [3]:
# tc = tornet.catalog()
# tc.to_csv(f'{TORNET_ROOT}/catalog.csv')

In [ ]:
import keras
from tornet.data.loader import get_dataloader
from huggingface_hub import hf_hub_download
from tornet.models.keras.layers import CoordConv2D
from tensorflow.keras.metrics import AUC, Precision, Recall

# Download the model
model_path = hf_hub_download(
    repo_id="tornet-ml/tornado_detector_baseline_v1",
    filename="tornado_detector_baseline.keras"
)

# Load model
model = keras.saving.load_model(
    model_path,
    custom_objects={'CoordConv2D': CoordConv2D},
    compile=False
)

# Use their loader!
ds = get_dataloader(
    dataloader='keras',
    data_root=TORNET_ROOT,
    years=YEARS,
    data_type='test',
    batch_size=32,
    select_keys=list(model.input.keys())
)

# Evaluate
model.compile(
    loss="binary_crossentropy",
    metrics=["accuracy", AUC(name="auc"), Precision(name="precision"), Recall(name="recall")]
)

results = model.evaluate(ds)

2025-04-26 01:30:05.303982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745631005.324440 4180980 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745631005.330814 4180980 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745631005.347533 4180980 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745631005.347550 4180980 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745631005.347553 4180980 computation_placer.cc:177] computation placer alr

672/672 ━━━━━━━━━━━━━━━━━━━━ 730s 1s/step - accuracy: 0.9546 - auc: 0.6926 - loss: 0.6178 - precision: 0.5631 - recall: 0.3304


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np 

metrics = dict(zip(model.metrics_names, results))
print(metrics)

# Predict labels
y_pred_probs = model.predict(ds)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# True labels
y_true = np.concatenate([y for x, y in ds], axis=0)

# Confusion Matrix and F1
cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(classification_report(y_true, y_pred, digits=4))
print("Confusion Matrix:\n", cm)
print("F1 Score:", f1)

{'loss': 0.6051602959632874, 'compile_metrics': 0.9551523327827454}
672/672 ━━━━━━━━━━━━━━━━━━━━ 710s 1s/step
              precision    recall  f1-score   support

           0     0.9673    0.9863    0.9767     20455
           1     0.5596    0.3433    0.4255      1040

    accuracy                         0.9552     21495
   macro avg     0.7634    0.6648    0.7011     21495
weighted avg     0.9475    0.9552    0.9500     21495

Confusion Matrix:
 [[20174   281]
 [  683   357]]
F1 Score: 0.42550655542312277
